# Obtener non-word characters únicos en una columna específica – Pytd & Regexp 


- Este programa de python, combina pytd y un query de expresiones regulares que obtiene una lista única de los caracteres especiales que no son parte de una palabra.  

- La salida es una lista de valores únicos. 

- Este caso puede ser ocupado para indentificar, tratar y remplazar los casos excepcionales. Por ejemplo, en la validación de nombres, agrupando casos por caracteres especiales. 

- La expresión regular niega los caracteres comunes para un nombre.  

## Importación & configuración

In [39]:
import pytd
import pandas as pd
client = pytd.Client(
apikey = '', #Insert your enviroment's API key.
database='src',
default_engine='presto'
)

## Queries de expresiones regulares & transformación de resultados a DataFrames

In [40]:
#Define the name of the table that you want to analyze

table_name = 'nvdr_history'
column_name = 'nombre_cte'

#This query gets the number of records with non-word characters
#We will use these special characters as part of our regular expression when the enconding format be solved. We will exclude them.
#\á\é\í\ó\ú\Á\É\Í\Ó\Ú\ü\Ü\ñ\Ñ

res0= client.query(
    f"""
    select count(distinct ({column_name})) as unique_non_word_records
    from {table_name}
    where REGEXP_LIKE({column_name},'^[a-zA-Z\s]*$') = False

    """,engine = 'presto'
)
df0 = pd.DataFrame(**res0) 

#This query gets a list of lists where we have all non-word characters
#This results gets all the patterns and all possible values.

#We will use these special characters as part of our regular expression when the enconding format be solved. We will exclude them.
#\á\é\í\ó\ú\Á\É\Í\Ó\Ú\ü\Ü\ñ\Ñ

res = client.query(
    f"""
    select distinct REGEXP_EXTRACT_ALL({column_name}, '[^a-zA-Z\s]') as other
    from {table_name}
    order by other asc

    """,engine = 'presto'
)
df = pd.DataFrame(**res)

print(df.head()) #The list of lists for non_characters frequency

          other
0            []
1        [, ]
2           ["]
3        [", "]
4  [", ", ", "]


## Extracción & definición de listas de caracteres especiales

In [41]:
#In this part we are going to unpack the values only in one list
# This works for regular nested list with list comprehension.
# https://appdividend.com/2022/06/17/how-to-flatten-list-in-python/

flatten_list = [element for sublist in df['other'] for element in sublist]
Special_characters = list(set(flatten_list)) #Getting the unique values into a single list.
Special_characters.sort()

#Finding missing values in the special characters list.

Missing_characters = ['á','é','í','ó','ú','Á','É','Í','Ó','Ú','ü','Ü','ñ','Ñ']
Found_characters = [value for value in Missing_characters if value in Special_characters]



## DataFrame: Total de records por cada caracter especial

In [42]:
#This query retrieves the distinct records from the original column.

res2 = client.query(
    f"""
    select distinct {column_name} as nombre
    from {table_name}
    order by nombre asc

    """,engine = 'presto'
)
df2 = pd.DataFrame(**res2)



frequency_list = [] #The list is used to store all the values.
for i in Special_characters:
    validation_list = [x for x in df2['nombre'] if i in x ]
    frequency_list.append(len(validation_list))

dict = {'s_character': Special_characters, 'Total of records': frequency_list} #The dictionary relates the columns with the lists.
characters_summary = pd.DataFrame(dict) #Creates the DataFrame from the dict.
characters_summary.sort_values(by='Total of records', ascending = False, inplace=  True) #Sorts the values.
characters_summary.reset_index(drop=True, inplace= True) # Resets the index from 0 to the length of the DF.


## Presentación de resultados

In [43]:
#Here we have the special characters in a sorted way.
print('NOTE: When the enconding format be solved, we will use these special characters as part of our regular expression to exclude them: \á\é\í\ó\ú\Á\É\Í\Ó\Ú\ü\Ü\ñ\Ñ')
print('\n')
print(f'1) Number of unique non_word characters records: {df0.iloc[0][0]}')
print(f'2) Length of the special characters list: {len(Special_characters)}')
print(f'3) List of non_word characters: {Special_characters}')
print(f'4) Found characters: {Found_characters}')
print('5) Characters summary')
characters_summary




NOTE: When the enconding format be solved, we will use these special characters as part of our regular expression to exclude them: \á\é\í\ó\ú\Á\É\Í\Ó\Ú\ü\Ü\ñ\Ñ


1) Number of unique non_word characters records: 29140
2) Length of the special characters list: 36
3) List of non_word characters: ['\x1a', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '@', '_', '`', '|', '}', 'Ƭ', '‡', '‹', '�']
4) Found characters: []
5) Characters summary


,s_character,Total of records
0,�,15387
1,/,9424
2,.,4942
3,",",2762
4,&,323
5,-,187
6,2,105
7,1,100
8,3,85
9,‹,82
